In [2]:
import numpy as np
import pandas as pd

In [4]:
k = np.load("../../baseline/rnn-nested-cv/2/0/valid-error-0-2-GRU-100-1-False-0.01-1000-0.0-['dr'].npy")

In [4]:
valid_error = {}
for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
    valid_error[appliance] = {}
    for fold_num in range(5):
        valid_error[appliance][fold_num] = {}

        for cell_type  in ['GRU', 'LSTM', 'RNN']:
            valid_error[appliance][fold_num][cell_type] = {}
            for hidden_size in [20, 50, 100]:
                valid_error[appliance][fold_num][cell_type][hidden_size] = {}
                for num_layers in range(1, 5):
                    valid_error[appliance][fold_num][cell_type][hidden_size][num_layers] = {}
                    for bidirectional in [True, False]:
                        valid_error[appliance][fold_num][cell_type][hidden_size][num_layers][bidirectional] = {}
                        for lr in [0.01, 0.1, 1.0]:
                            valid_error[appliance][fold_num][cell_type][hidden_size][num_layers][bidirectional][lr] = {}
                            for iters in [1000, 2000, 3000]:
                                filename = "valid-error-{}-3-{}-{}-{}-{}-{}-{}-0.0-[\'{}\'].npy".format(fold_num, 
                                                                                                    cell_type,
                                                                                                   hidden_size,
                                                                                                   num_layers,
                                                                                                   bidirectional,
                                                                                                   lr, iters, appliance)
                                directory = "../../baseline/rnn-nested-cv/3/{}/".format(fold_num)
    #                             print(filename)
                                full_path = directory + filename
                                valid_error[appliance][fold_num][cell_type][hidden_size][num_layers][bidirectional][lr][iters] = np.load(full_path).item()[appliance]


FileNotFoundError: [Errno 2] No such file or directory: "../../baseline/rnn-nested-cv/3/0/valid-error-0-3-GRU-20-1-True-0.01-1000-0.0-['hvac'].npy"

In [29]:
valid_error['hvac'][0]['GRU'][20][1][False][0.01][1000]

612.27177617977202

In [3]:
best_param = {}
best_error = {}
for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
    best_param[appliance] = {}
    best_error[appliance] = {}
    for fold_num in range(5):
        best_param[appliance][fold_num] = {}
        best_error[appliance][fold_num] = np.Inf
        for cell_type  in ['GRU', 'LSTM', 'RNN']:
            for hidden_size in [20, 50, 100]:
                for num_layers in range(1, 5):
                    for bidirectional in [True, False]:
                        for lr in [0.01, 0.1, 1.0]:
                            for iters in [1000, 2000, 3000]:
                                cur_error = valid_error[appliance][fold_num][cell_type][hidden_size][num_layers][bidirectional][lr][iters]
                                if best_error[appliance][fold_num] > cur_error:
                                    best_error[appliance][fold_num] = cur_error
                                    best_param[appliance][fold_num]['cell_type'] = cell_type
                                    best_param[appliance][fold_num]['hidden_size'] = hidden_size
                                    best_param[appliance][fold_num]['num_layers'] = num_layers
                                    best_param[appliance][fold_num]['bidirectional'] = bidirectional
                                    best_param[appliance][fold_num]['lr'] = lr
                                    best_param[appliance][fold_num]['iters'] = iters

In [5]:
np.save("../code/baseline/result/rnn-individual-param-2.npy", best_param)

In [44]:
for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
    for fold_num in range(5):
        cell_type = best_param[appliance][fold_num]['cell_type'] 
        hidden_size = best_param[appliance][fold_num]['hidden_size'] 
        num_layers = best_param[appliance][fold_num]['num_layers'] 
        bidirectional = best_param[appliance][fold_num]['bidirectional'] 
        lr = best_param[appliance][fold_num]['lr'] 
        iters = best_param[appliance][fold_num]['iters'] 
        print("python rnn-nested-cv-individual.py 2 {} {} {} {} {} {} 0 {} {}".format(cell_type, hidden_size, num_layers, bidirectional, lr, iters, fold_num, appliance))

python rnn-nested-cv-individual.py 2 LSTM 50 2 True 0.01 3000 0 0 hvac
python rnn-nested-cv-individual.py 2 LSTM 100 2 True 0.01 2000 0 1 hvac
python rnn-nested-cv-individual.py 2 LSTM 100 2 True 0.01 3000 0 2 hvac
python rnn-nested-cv-individual.py 2 LSTM 100 2 True 0.01 3000 0 3 hvac
python rnn-nested-cv-individual.py 2 LSTM 100 2 True 0.01 3000 0 4 hvac
python rnn-nested-cv-individual.py 2 GRU 100 3 True 0.01 1000 0 0 fridge
python rnn-nested-cv-individual.py 2 LSTM 100 1 True 0.01 3000 0 1 fridge
python rnn-nested-cv-individual.py 2 RNN 50 2 True 0.01 2000 0 2 fridge
python rnn-nested-cv-individual.py 2 LSTM 20 2 True 0.01 3000 0 3 fridge
python rnn-nested-cv-individual.py 2 LSTM 20 4 True 0.01 1000 0 4 fridge
python rnn-nested-cv-individual.py 2 GRU 20 1 True 0.1 3000 0 0 dr
python rnn-nested-cv-individual.py 2 GRU 100 2 True 0.01 2000 0 1 dr
python rnn-nested-cv-individual.py 2 RNN 50 2 True 0.01 3000 0 2 dr
python rnn-nested-cv-individual.py 2 GRU 20 1 True 0.1 2000 0 3 dr
pytho

In [51]:
test_error = {}
for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
    test_error[appliance] = {}
    for fold_num in range(5):
        cell_type = best_param[appliance][fold_num]['cell_type'] 
        hidden_size = best_param[appliance][fold_num]['hidden_size'] 
        num_layers = best_param[appliance][fold_num]['num_layers'] 
        bidirectional = best_param[appliance][fold_num]['bidirectional'] 
        lr = best_param[appliance][fold_num]['lr'] 
        iters = best_param[appliance][fold_num]['iters']
        
        directory = "../../baseline/rnn-individual/2/{}/{}/{}/{}/{}/{}/{}/0.0/".format(fold_num, cell_type,
                                                                                                hidden_size,
                                                                                                num_layers,
                                                                                                bidirectional,
                                                                                                lr, iters)
        filename = "test-error-{}-2-{}-{}-{}-{}-{}-{}-0.0-[\'{}\'].npy".format(fold_num, cell_type, hidden_size, num_layers,
                                                         bidirectional, lr, iters, appliance)
        full_path = directory + filename
        test_error[appliance][fold_num] = np.load(full_path).item()[iters][appliance]

In [54]:
pd.DataFrame(test_error).T

,0,1,2,3,4
dr,51.326346,64.961606,71.062742,83.048024,65.399230
dw,11.061491,16.492105,19.615092,20.592083,9.510532
fridge,27.452612,34.600016,25.526234,31.120937,29.966619
hvac,418.030461,404.558100,352.350371,544.693777,477.629806
mw,6.766763,10.059784,6.491012,5.547876,12.538048
